In [1]:

import os
os.chdir('../')
%pwd

'/home/jafarid/code/yogaposes'

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    resnet_base_model_path: Path
    resnet_updated_base_model_path: Path
    params_class: int
    params_image_size: list
    params_pretrained: bool

In [35]:
from yogaposes.constants import *
from yogaposes.utils.common import read_yaml, create_directories


In [47]:
class ConfigurationManager: 
    def __init__(self, config_file_path= CONFIG_FILE_PATH, params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        get_prepare_base_model_config = PrepareBaseModelConfig(root_dir= config.root_dir,
                                                               resnet_base_model_path=config.resnet_base_model_path,
                                                               resnet_updated_base_model_path= config.resnet_updated_base_model_path,
                                                               params_class=self.params.CLASSES,
                                                               params_image_size= self.params.IMAGE_SIZE,
                                                               params_pretrained = self.params.PRETRAINED
                                                               )

        return get_prepare_base_model_config
    

In [5]:
import os
from torchvision import models
import torch 
from torchsummary import summary
from yogaposes import logger 

In [7]:

full_model = models.resnet18(pretrained=True)
summary(full_model, input_size=tuple([3,244,244]), batch_size=15, device='cuda')


/home/jafarid/miniconda3/envs/yogaposes/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jafarid/miniconda3/envs/yogaposes/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/jafarid/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.7MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [15, 64, 122, 122]           9,408
       BatchNorm2d-2         [15, 64, 122, 122]             128
              ReLU-3         [15, 64, 122, 122]               0
         MaxPool2d-4           [15, 64, 61, 61]               0
            Conv2d-5           [15, 64, 61, 61]          36,864
       BatchNorm2d-6           [15, 64, 61, 61]             128
              ReLU-7           [15, 64, 61, 61]               0
            Conv2d-8           [15, 64, 61, 61]          36,864
       BatchNorm2d-9           [15, 64, 61, 61]             128
             ReLU-10           [15, 64, 61, 61]               0
       BasicBlock-11           [15, 64, 61, 61]               0
           Conv2d-12           [15, 64, 61, 61]          36,864
      BatchNorm2d-13           [15, 64, 61, 61]             128
             ReLU-14           [15, 64,

/home/jafarid/miniconda3/envs/yogaposes/lib/python3.8/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
full_model.fc.in_features
full_model.fc.out_features

512

In [52]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
    def get_base_model(self):
        resnet_model = models.resnet18(pretrained=self.config.params_pretrained)
        resnet_model.to(self.device)
        self.save_model(resnet_model, self.config.resnet_base_model_path)
        return resnet_model
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_till, freeze_all=True):
    # internal function that we don't want to run
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
                
        elif (freeze_till is not None) and (freeze_till>0):
            for param in model.parameters()[:-freeze_till]:
                param.requires_grad = False
                
        n_inputs = model.fc.in_features
        model.fc = torch.nn.Linear(n_inputs, classes)
        return model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(model= self.get_base_model(), classes=self.config.params_class, freeze_all=True, freeze_till=None)
        self.full_model.to(self.device)
        summary(self.full_model,input_size=tuple(self.config.params_image_size), device=self.device)
        
        self.save_model(checkpoint=self.full_model, path=self.config.resnet_updated_base_model_path)
        logger.info(f'Saved updated model to {str(self.config.root_dir)}')

    
    @staticmethod
    def save_model(checkpoint:dict,path:Path):
        torch.save(checkpoint, path)
        

In [54]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e

[2024-03-02 17:54:21,305: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-02 17:54:21,306: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-02 17:54:21,307: INFO: common: created directory at: artifacts]
[2024-03-02 17:54:21,307: INFO: common: created directory at: artifacts/prepare_base_model]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-